In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd

In [5]:
from scr.util import *

# Base Infomation

In [6]:
df = pd.read_csv("data/train.csv")

In [4]:
df.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken
0,0,50歳,Self Enquiry,2,900秒,Large Business,male,1.0,4.0,Basic,3.0,5,1,4,Executive,253905.0,未婚 車未所持 子供なし,1
1,1,56歳,Company Invited,1,14分,Salaried,Male,1.0,4.0,Standard,3.0,2,1,4,Senior Manager,404475.0,"離婚済み,車あり,子供無し",0
2,2,NaN,Self Enquiry,1,10分,Large Business,Female,1.0,3.0,Basic,3.0,4,0,4,Executive,278145.0,"結婚済み、自動車未所有,子供なし",1
3,3,三十七歳,Self Enquiry,2,1080秒,Small Business,female,1.0,3.0,Standard,4.0,1,0,5,Senior Manager,326805.0,離婚済み、車所持、子供無し,0
4,4,48歳,Company Invited,3,1020秒,Small Business,female,1.0,3.0,Basic,4.0,4,0,4,Executive,258435.0,独身／車所持／無子,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   Age                     3389 non-null   object 
 2   TypeofContact           3483 non-null   object 
 3   CityTier                3489 non-null   int64  
 4   DurationOfPitch         3368 non-null   object 
 5   Occupation              3489 non-null   object 
 6   Gender                  3489 non-null   object 
 7   NumberOfPersonVisiting  3489 non-null   float64
 8   NumberOfFollowups       3456 non-null   float64
 9   ProductPitched          3489 non-null   object 
 10  PreferredPropertyStar   3489 non-null   float64
 11  NumberOfTrips           3467 non-null   object 
 12  Passport                3489 non-null   int64  
 13  PitchSatisfactionScore  3489 non-null   int64  
 14  Designation             3489 non-null   

In [6]:
df.describe()

,id,CityTier,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,Passport,PitchSatisfactionScore,ProdTaken
count,3489.000000,3489.000000,3489.000000,3456.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,1744.000000,1.697048,2.394669,7.765336,3.441101,0.102608,2.513901,0.142448
std,1007.331872,0.671500,0.726612,39.498670,0.599633,0.303490,1.115412,0.349559
min,0.000000,1.000000,1.000000,1.000000,3.000000,0.000000,1.000000,0.000000
25%,872.000000,1.000000,2.000000,3.000000,3.000000,0.000000,2.000000,0.000000
50%,1744.000000,2.000000,2.000000,4.000000,3.000000,0.000000,2.000000,0.000000
75%,2616.000000,2.000000,3.000000,4.000000,4.000000,0.000000,3.000000,0.000000
max,3488.000000,3.000000,4.000000,600.000000,5.000000,1.000000,5.000000,1.000000


In [7]:
df.isna().sum()

id                          0
Age                       100
TypeofContact               6
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          33
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
customer_info               0
ProdTaken                   0
dtype: int64

In [16]:
tmp = ["TypeofContact", "Occupation", "Gender", "ProductPitched", "NumberOfTrips", "Designation", "customer_info"]
for column in tmp:
    print(column)
    print(df[column].unique(), "\n")

TypeofContact
['Self Enquiry' 'Company Invited' nan] 

Occupation
['Large Business' 'Salaried' 'Small Business'] 

Gender
['male' 'Male' 'Female' 'female' 'ｆｅｍａｌｅ' 'FEMALE' 'Ｍａｌｅ' 'Fe Male' 'MALE'
 'ｆｅ\u3000ｍａｌｅ' 'fe male' 'Ｆｅｍａｌｅ' 'ｍａｌｅ' 'FE MALE' 'Ｆｅ\u3000Ｍａｌｅ'
 'ＦＥＭＡＬＥ' 'ＦＥ\u3000ＭＡＬＥ' 'ＭＡＬＥ'] 

ProductPitched
['Basic' 'Standard' 'Super Deluxe' 'basic' 'SUPER DELUXE' 'super deluxe'
 'BASIC' 'Deluxe' 'deluxe' 'STANᗞARD' 'STANDARD' 'Вasic' 'DELUXE'
 'ꓢuper De|uxe' 'Ѕuper Deluxe' 'BAՏIC' 'Basıc' 'King' 'Super De|uxe'
 'king' 'standard' 'KING' 'BΑSIC' 'B𝖺sic' 'De|uxe' 'ᎠELUXE' 'Basiϲ'
 'de|u×e' 'Delu×e' 'Standar𝔡' 'Basi𝘤' 'Βasic' 'Տuper Deluxe' 'Staոdard'
 'BAꓢIC' 'ᗞeluxe' 'Տtandard' 'Βası𝘤' 'Kıng' 'Baｓic' 'basıc' 'super de|uxe'
 'Stand𝖺rd' 'S𝘵andard' '𐊡asic' 'St𝖺ndard' 'Super ᗞeluxe' 'de|uxe'
 'ｓuper deluxe' 'STANDARᎠ' 'Basiс' 'DΕLUXΕ' 'ꓢuper Deluxe' 'BASΙC'
 'ꓢtandard' 'В𝖺sic' 'Standa𝘳d' 'basiϲ' 'staոdard' 'Super Ꭰeluxe' 'DELUXΕ'
 'Ѕtandard' '𐊡asi𝘤' 'Ꭰeluxe' 'Kıոg' '𝙳eluxe' 'Kiոg' 'Βas

# Processing

## NuLL
・fill mediam : '', '', '', 'NumberOfTrips', 'MonthlyIncome'
    
・delete null : 'TypeofContact'

### Age -> age

In [33]:
df["Age"].unique()

array(['50歳', '56歳', nan, '三十七歳', '48歳', '19歳', '47歳', '49歳', '33歳',
       '３１歳', '51歳', '46歳', '４９歳', '53才', '20歳', '25歳', '五十三歳', '20代',
       '44歳', '30歳', '34歳', '29歳', '30代', '41歳', '50代', '52歳', '22歳',
       '３６歳', '28歳', '25才', '五十一歳', '48才', '38才', '三十三歳', '42際', '５０代',
       '35才', '59歳', '四十三歳', '32歳', '４２才', '53歳', '58歳', '40代', '二十六歳',
       '52際', '54歳', '59才', '29才', '21才', '57歳', '37歳', '35歳', '36才',
       '38歳', '３７歳', '50才', '51才', '43歳', '四十歳', '26歳', '31歳', '27歳',
       '40歳', '３４歳', '40際', '21歳', '24才', '45歳', '36歳', '41才', '38際',
       '18歳', '39歳', '４６歳', '４０代', '５５歳', '58際', '24歳', '41際', '三十一歳',
       '52才', '３０代', '28才', '55際', '60代', '27際', '３２歳', '60歳', '五十四歳',
       '29際', '三十四歳', '４７歳', '37際', '二十四歳', '四十七歳', '58才', '39才', '23才',
       '３３歳', '４２歳', '４５歳', '３９歳', '５４歳', '55歳', '42歳', '３８歳', '47才',
       '５１歳', '３９才', '51際', '二十七歳', '30才', '２６歳', '４０歳', '36際', '三十歳',
       '23歳', '二十二歳', '37才', '３０歳', '２８歳', '34才', '五十歳', '四十一歳', '５９歳',
       '

In [7]:
# 日本語をすべてInt型に変換
df["age"] = df["Age"].astype(str).apply(japanese_to_int)

# 欠損値を中央値で補完
# japanese_to_int関数により、欠損値は0と出力されている。
tmp = df[~(df["age"] == 0)]
df.loc[df["age"] == 0, "age"] = tmp["age"].median()

In [11]:
df["age"].unique()

array([50, 56, 38, 37, 48, 19, 47, 49, 33, 31, 51, 46, 53, 20, 25, 44, 30,
       34, 29, 41, 52, 22, 36, 28, 42, 35, 59, 43, 32, 58, 40, 26, 54, 21,
       57, 27, 24, 45, 18, 39, 55, 60, 23, 10, 61], dtype=int64)

### DurationOfPitch -> duration

In [8]:
df["DurationOfPitch"].unique()

array(['900秒', '14分', '10分', '1080秒', '1020秒', '16分', '840秒', '22分',
       '20分', '11分', '32分', '17分', '24分', '1380秒', '35分', '420秒', '5分',
       '480秒', '8分', '15分', '960秒', nan, '540秒', '26分', '13分', '12分',
       '21分', '25分', '720秒', '9分', '7分', '780秒', '6分', '18分', '31分',
       '600秒', '23分', '28分', '2160秒', '660秒', '1500秒', '1920秒', '27分',
       '33分', '360秒', '1320秒', '1740秒', '1680秒', '29分', '1620秒', '1440秒',
       '1800秒', '30分', '34分', '4分', '19分', '1860秒', '1260秒', '2100秒',
       '300秒', '2040秒', '1200秒', '1560秒', '1980秒', '36分', '1140秒'],
      dtype=object)

In [16]:
# 単位をすべて秒に統一
df["duration"] = df["DurationOfPitch"].dropna().apply(convert_to_second)

# 欠損値を中央値で補完
df["duration"] = df["duration"].fillna(df["duration"].median())

In [17]:
df["duration"].unique()

array([ 900.,  840.,  600., 1080., 1020.,  960., 1320., 1200.,  660.,
       1920., 1440., 1380., 2100.,  420.,  300.,  480.,  780.,  540.,
       1560.,  720., 1260., 1500.,  360., 1860., 1680., 2160., 1620.,
       1980., 1740., 1800., 2040.,  240., 1140.])

### NumberOfFollowups